In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut
import matplotlib.pyplot as plt
import seaborn as sb  
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split  
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing  import StandardScaler
import joblib
import warnings
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.visualization.scatter import Scatter
from pymoo.optimize import minimize
warnings.filterwarnings("ignore")
df=pd.read_excel('MD-15.xlsx')
X=df.iloc[:,1:8]
y=df.iloc[:,8]

In [ ]:
# Grid search for training GBR model

In [ ]:
best_score =0
list_1=np.arange(0,21,1)#   
list_2=np.arange(1,20,1)#  
list_3=np.arange(1,200,5) 
for a in list_1:
    for b in list_2:
        for c in list_3:
            X_train,X_test= train_test_split(X, test_size=0.2,random_state = 0)
            y_train,y_test = train_test_split(y, test_size=0.2,random_state = 0)
            sc = StandardScaler()
            X_train_std = sc.fit_transform(X_train)
            X_test_std = sc.transform(X_test)
            gbrt = GradientBoostingRegressor(random_state=a,n_estimators=c,max_depth=b)
            gbrt.fit(X_train_std, y_train)
            y_train_pred_gbrt = gbrt.predict(X_train_std)  
            y_test_pred_gbrt =gbrt.predict(X_test_std)
            score=r2_score(y_test,y_test_pred_gbrt)
            if score > best_score:
                best_score = score
                best_parameters = {'random_state': a, 'learning_rate': b,'n_estimators': c}
print("Best score: {:.5f}".format(best_score))
print("Best parameters: {}".format(best_parameters))  

In [ ]:
#Feature importance analysis of the GBR model

In [ ]:
X_train,X_test= train_test_split(X, test_size=0.2,random_state =0)
y_train,y_test = train_test_split(y, test_size=0.2,random_state =0)
from sklearn.preprocessing  import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(random_state=0,n_estimators=46,max_depth=4,max_features=8)
gbrt.fit(X_train_std, y_train)
feature_importances = gbrt.feature_importances_
feature_importance_df = pd.DataFrame({
    'Importance': feature_importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
#Leave-one-out cross-validation

In [ ]:
gbrt = GradientBoostingRegressor()
loo = LeaveOneOut()
mae_list = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mae_list.append(mae)
mae_mean = np.mean(mae_list)
mae_variance = np.var(mae_list)

In [ ]:
# Multi-objective optimization

In [ ]:
t1=joblib.load('gbri1.joblib')
t2=joblib.load('gbri2.joblib')
t3=joblib.load('gbri3.joblib')
t4=joblib.load('gbri4.joblib')
t5=joblib.load('gbri5.joblib')
t6=joblib.load('gbri6.joblib')
X=df.iloc[:,1:8]
X=X.values
sc = StandardScaler()
X= sc.fit_transform(X)
class MyProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=3, 
                         n_obj=6,  
                         n_ieq_constr=2, 
                         xl=np.array([0.2, 0.3, 0]),  
                         xu=np.array([0.4, 0.7, 0.5]))  
    def _evaluate(self, x, out, *args, **kwargs):
        g1 = x[0]+x[1]+x[2]-1
        g2 = 0.5-(x[0]+x[1]+x[2])
        a=1-x[0]-x[1]-x[2]       
        x2=x[2]/(x[2]+a)
        x3=x[1]/(x[0]+x[2]+a)
        x1=x[0]/(x[1]+x[2]+a)
        X=[x[0],x[1],x[2],a,x1,x2,x3]
        x=np.array(X)
        x=sc.transform(x.reshape(1, -1))
        f1 = - 10**t1.predict(x.reshape(1, -1)) 
        f2 = - 10**t2.predict(x.reshape(1, -1))
        f3 = - 10**t3.predict(x.reshape(1, -1))
        f4 = - 100*t4.predict(x.reshape(1, -1))
        f5 = - 100*t5.predict(x.reshape(1, -1))
        f6 = - 100*t6.predict(x.reshape(1, -1))
        out["F"] = np.array([f1,f2,f3,f4,f5,f6])
        out["G"] = np.array([g1,g2])
problem = MyProblem()
ref_dirs = get_reference_directions("das-dennis",6, n_partitions=6)
termination = get_termination("n_gen", 500)
algorithm = NSGA3(ref_dirs=ref_dirs,pop_size=500, 
                  n_offsprings=1000,
                  sampling=FloatRandomSampling(),
                  termination=termination,
                  crossover=SBX(prob=0.8, eta=15),
                  mutation=PM(eta=20),
                  eliminate_duplicates=True)
res = minimize(problem,
               algorithm,
               seed=1,
               save_history=True,
               verbose=True)
X = res.X
F = res.F